In [8]:
# Initial State and Goal for a Block World Problem
initial_state = {
    'on_table': {'A', 'C'},
    'clear': {'B', 'C'},
    'on': {'B': 'A'}
}

goal_state = {
    'on': {'A': 'B', 'B': 'C'}
}

# Stack for storing goals
goal_stack = []

# Actions and operations for manipulating blocks
def is_goal_satisfied(state, goal):
    """ Check if the current state satisfies the given goal """
    for key, value in goal.items():
        if key == 'on':
            for block, under in value.items():
                if state.get('on', {}).get(block) != under:
                    return False
    return True

def apply_action(state, action):
    """ Apply an action to change the state """
    action_type, *params = action
    
    if action_type == 'unstack':
        block, from_block = params
        state['clear'].add(from_block)
        state['clear'].remove(block)
        state['on'].pop(block)
        
    elif action_type == 'stack':
        block, onto_block = params
        state['on'][block] = onto_block
        
        # Update clear set correctly
        state['clear'].add(onto_block)  # Now the block on which it is stacked is clear
        state['clear'].remove(block)    # The block that was stacked is no longer clear
        
    elif action_type == 'pickup':
        block = params[0]
        state['clear'].remove(block)
        
    elif action_type == 'putdown':
        block = params[0]
        state['on_table'].add(block)
        state['clear'].add(block)

def goal_stack_planner(initial_state, goal_state):
    """ Goal Stack Planning algorithm for the Block World """
    # Start with the initial goal
    goal_stack.append(goal_state)
    
    # Make a copy of the initial state
    current_state = initial_state.copy()
    current_state['on'] = current_state['on'].copy()
    current_state['clear'] = current_state['clear'].copy()
    current_state['on_table'] = current_state['on_table'].copy()
    
    while goal_stack:
        goal = goal_stack.pop()
        
        # If the goal is an action, apply it
        if isinstance(goal, tuple) and goal[0] in {'stack', 'unstack', 'pickup', 'putdown'}:
            apply_action(current_state, goal)
            print(f"Action: {goal}")
            print(f"Current State: {current_state}")
        
        # If the goal is a condition, check if it's satisfied
        elif isinstance(goal, dict):
            if is_goal_satisfied(current_state, goal):
                continue
            else:
                # Break the goal down into subgoals
                for block, under in goal.get('on', {}).items():
                    # Subgoal: make sure block is clear
                    if block not in current_state['clear']:
                        goal_stack.append(('unstack', block, current_state['on'].get(block, None)))
                    
                    # Subgoal: make sure under is clear
                    if under not in current_state['clear']:
                        goal_stack.append(('unstack', under, current_state['on'].get(under, None)))
                    
                    # Subgoal: stack block onto under
                    goal_stack.append(('stack', block, under))
                
    print("\nFinal State:")
    print(current_state)

# Example Usage
goal_stack_planner(initial_state, goal_state)


Action: ('stack', 'B', 'C')
Current State: {'on_table': {'A', 'C'}, 'clear': {'C'}, 'on': {'B': 'C'}}


KeyError: 'A'